# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

Answer: There are 41188 marketing campaigns in the full dataset. 

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [112]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import time
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import recall_score
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures

In [3]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [5]:
# need to drop duration in order to build a realistic predictive model
df = df.drop(columns = 'duration')

In [6]:
# map y to numeric
df.y = df.y.map({'yes': 1, 'no': 0})

In [11]:
# here we can see there are no nulls. some columns contain 'unknown' as values which I could consider dropping, but I will keep in order
# to preserve maximum data 
df.isna().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

Business Objective: Based on the various information about marketing campaigns for this Portuguese banking institution, we want to build a model to predict whether or not a marketing campaign will lead to a bank term deposit. In order to do this, we will build and assess various models and choose the best one. We will use accuracy as the metric for scoring, because in this case there is no outsized harm for false negatives or false positives, so it's better to be generally accurate. 

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  campaign        41188 non-null  int64  
 11  pdays           41188 non-null  int64  
 12  previous        41188 non-null  int64  
 13  poutcome        41188 non-null  object 
 14  emp.var.rate    41188 non-null  float64
 15  cons.price.idx  41188 non-null  float64
 16  cons.conf.idx   41188 non-null  float64
 17  euribor3m       41188 non-null 

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [17]:
# Note, I am not sure why or what is meant by "just the bank information features", as all of the features are information about related to 
# the bank's marketing campaigns, so I am going to assume I can use all of them. 
X = df.drop(columns = 'y')
y = df['y']

In [25]:
non_numeric_columns = np.array(X.select_dtypes(include='object').columns.values)
transformer = make_column_transformer((OneHotEncoder(), non_numeric_columns), remainder = StandardScaler())
X_transformed = transformer.fit_transform(X)
X_transformed = pd.DataFrame(X_transformed, columns = transformer.get_feature_names_out())

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size = 0.3)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [32]:
# We can use Dummy Classifier for this
dummy = DummyClassifier().fit(X_train, y_train)
dummy_train_score = dummy.score(X_train, y_train)
dummy_test_score = dummy.score(X_test, y_test)
print("The Dummy Classifier helps us establish a baseline. The train score is " + str(dummy_train_score)
      + " and the test score is " + str(dummy_test_score))

The Dummy Classifier helps us establish a baseline. The train score is 0.886025458707641 and the test score is 0.8904264789188314


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [39]:
# Train & time model
lr_start_time = time.time()
lr = LogisticRegression().fit(X_train, y_train)
lr_end_time = time.time()

# Calculate & store metrics
lr_train_score = lr.score(X_train, y_train)
lr_test_score = lr.score(X_test, y_test)
lr_train_time = lr_end_time - lr_start_time

### Problem 9: Score the Model

What is the accuracy of your model?

In [40]:
print("Logistic regression has a train score of " + str(lr_train_score) + " and a test score of " + str(lr_test_score))

Logistic regression has a train score of 0.8998647289376018 and a test score of 0.9019179412478757


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [41]:
# KNN 

# Train & time model
knn_start_time = time.time()
knn = KNeighborsClassifier().fit(X_train, y_train)
knn_end_time = time.time()

# Calculate & store metrics
knn_train_score = knn.score(X_train, y_train)
knn_test_score = knn.score(X_test, y_test)
knn_train_time = knn_end_time - knn_start_time

In [42]:
# Decision Tree

# Train & time model
dt_start_time = time.time()
dt = DecisionTreeClassifier().fit(X_train, y_train)
dt_end_time = time.time()

# Calculate & store metrics
dt_train_score = dt.score(X_train, y_train)
dt_test_score = dt.score(X_test, y_test)
dt_train_time = dt_end_time - dt_start_time

In [43]:
# SVM

# Train & time model
svm_start_time = time.time()
svm = SVC().fit(X_train, y_train)
svm_end_time = time.time()

# Calculate & store metrics
svm_train_score = svm.score(X_train, y_train)
svm_test_score = svm.score(X_test, y_test)
svm_train_time = svm_end_time - svm_start_time

In [47]:
# Build DF

results_df = pd.DataFrame()
results_df['Model'] = ['Logistic Regression', 'K Neighbors', 'Decision Tree', 'SVM']
results_df['Train Time'] = [round(lr_train_time, 3), round(knn_train_time, 3), round(dt_train_time, 3), round(svm_train_time, 3)]
results_df['Train Accuracy'] = [round(lr_train_score, 3), round(knn_train_score, 3), round(dt_train_score, 3), round(svm_train_score, 3)]
results_df['Test Accuracy'] =  [round(lr_test_score, 3), round(knn_test_score, 3), round(dt_test_score, 3), round(svm_test_score, 3)]
results_df = results_df.set_index('Model')
results_df

,Train Time,Train Accuracy,Test Accuracy
Model,,,
Logistic Regression,0.230,0.900,0.902
K Neighbors,0.006,0.912,0.894
Decision Tree,0.160,0.996,0.837
SVM,23.250,0.905,0.901


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [87]:
# First I'll play around with feature engineering by using sequential feature selection to get down to just 5 top features
results_2 = pd.DataFrame()

In [88]:
# Redo logistic regression with feature selection

# Train & time model
lr_pipe_start_time = time.time()
lr_pipe = Pipeline([
    ('selector', SequentialFeatureSelector(LogisticRegression(), n_features_to_select = 5)),
    ('model', LogisticRegression())])
lr_pipe.fit(X_train, y_train)
lr_pipe_end_time = time.time()

# Calculate & store metrics
results_2['Model'] = ['Logistic Regression']
results_2['Train Time'] = [lr_pipe_end_time - lr_pipe_start_time]
results_2['Train Accuracy'] = [lr_pipe.score(X_train, y_train)]
results_2['Test Accuracy'] = [lr_pipe.score(X_test, y_test)]

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,18.642746,0.899657,0.899328


In [89]:
# Redo KNN with feature selection 

# Train & time model
knn_pipe_start_time = time.time()
knn_pipe = Pipeline([
    ('selector', SequentialFeatureSelector(LogisticRegression(), n_features_to_select = 5)),
    ('model', KNeighborsClassifier())])
knn_pipe.fit(X_train, y_train)
knn_pipe_end_time = time.time()

# Calculate & store metrics
results_2.loc[1] = ['K Neighbors', (knn_pipe_end_time - knn_pipe_start_time), knn_pipe.score(X_train, y_train),
                     knn_pipe.score(X_test, y_test)]


In [90]:
# Redo Decision Tree with feature Selection 
# Train & time model
dt_pipe_start_time = time.time()
dt_pipe = Pipeline([
    ('selector', SequentialFeatureSelector(LogisticRegression(), n_features_to_select = 5)),
    ('model',DecisionTreeClassifier())])
dt_pipe.fit(X_train, y_train)
dt_pipe_end_time = time.time()

# Calculate & store metrics
results_2.loc[2] = ['Decision Tree', (dt_pipe_end_time - dt_pipe_start_time), dt_pipe.score(X_train, y_train),
                     dt_pipe.score(X_test, y_test)]

In [92]:
# Redo SVM with feature selection 

# Train & time model
svm_pipe_start_time = time.time()
svm_pipe = Pipeline([
    ('selector', SequentialFeatureSelector(LogisticRegression(), n_features_to_select = 5)),
    ('model',SVC())])
svm_pipe.fit(X_train, y_train)
svm_pipe_end_time = time.time()

# Calculate & store metrics
results_2.loc[3] = ['SVM', (svm_pipe_end_time - svm_pipe_start_time), svm_pipe.score(X_train, y_train),
                     svm_pipe.score(X_test, y_test)]

In [94]:
results_2 = results_2.set_index('Model')
results_2

,Train Time,Train Accuracy,Test Accuracy
Model,,,
Logistic Regression,18.642746,0.899657,0.899328
K Neighbors,18.577167,0.895356,0.895201
Decision Tree,22.741363,0.901287,0.898762
SVM,21.391586,0.899483,0.900138


In [97]:
# Next I'll use GridSearch to optimize hyperparameters 
results_3 = pd.DataFrame()

In [98]:
# First LogisticRegression

# Train & time model 
lr_params = {'penalty' : ['l1', 'l2', 'elasticnet', None]}
lr_grid_start_time = time.time()
lr_grid = GridSearchCV(LogisticRegression(), param_grid = lr_params)
lr_grid.fit(X_train, y_train)
lr_grid_end_time = time.time()

# Calculate & store metrics
results_3['Model'] = ['Logistic Regression']
results_3['Train Time'] = [lr_grid_end_time - lr_grid_start_time]
results_3['Train Accuracy'] = [lr_grid.score(X_train, y_train)]
results_3['Test Accuracy'] = [lr_grid.score(X_test, y_test)]

In [101]:
# Then KNN

# Train & time model
knn_params = {'n_neighbors': [1, 2, 3, 4, 5], 'weights' : ['uniform', 'distance']}
knn_grid_start_time = time.time()
knn_grid = GridSearchCV(KNeighborsClassifier(), param_grid = knn_params)
knn_grid.fit(X_train, y_train)
knn_grid_end_time = time.time()

# Calculate and store metrics 
results_3.loc[1] = ['K Neighbors', (knn_grid_end_time - knn_grid_start_time), knn_grid.score(X_train, y_train),
                     knn_grid.score(X_test, y_test)]

In [105]:
# Decision Tree
dt_params = {'max_depth': [1, 2, 3, 4, 5]}

# Train & time model
dt_grid_start_time = time.time()
dt_grid = GridSearchCV(DecisionTreeClassifier(), param_grid = dt_params)
dt_grid.fit(X_train, y_train)
dt_grid_end_time = time.time()

# Calculate and store metrics
results_3.loc[2] = ['Decision Tree', (dt_grid_end_time - dt_grid_start_time), dt_grid.score(X_train, y_train),
                     dt_grid.score(X_test, y_test)]

In [108]:
# SVM - I have to comment this out, it's way too slow to run so this is not a feasible strategy
# svc_params = {'kernel': ['rbf', 'poly', 'linear', 'sigmoid'], 'gamma': [0.1, 1.0, 10.0, 100.0]}

# # Train & time model
# svc_grid_start_time = time.time()
# svc_grid = GridSearchCV(SVC(), param_grid = svc_params)
# svc_grid.fit(X_train, y_train)
# svc_grid_end_time = time.time()

# # Calculate and store metrics 
# results_3.loc[3] = ['SVM', (svc_grid_end_time - svc_grid_start_time), svc_grid.score(X_train, y_train),
#                      svc_grid.score(X_test, y_test)]

In [110]:
results_3 = results_3.set_index('Model')
results_3

,Train Time,Train Accuracy,Test Accuracy
Model,,,
Logistic Regression,1.930141,0.899691,0.902080
K Neighbors,7.668330,0.913669,0.896091
Decision Tree,0.636424,0.899830,0.900057


In [111]:
# Lastly, we will play around with adjusting our scoring. You could also argue that recall is a good metric here because
# for a marketing campaign, we want a model that is able to find all of the positive cases, because a false positive is much better
# than a false negative so we can actually make the most attempts at customers who will convert. 
results_4 = pd.DataFrame()

In [115]:
# Get scores
lr_train_recall = recall_score(y_train, lr.predict(X_train))
lr_test_recall = recall_score(y_test, lr.predict(X_test))
knn_train_recall = recall_score(y_train, knn.predict(X_train))
knn_test_recall = recall_score(y_test, knn.predict(X_test))
dt_train_recall = recall_score(y_train, dt.predict(X_train))
dt_test_recall = recall_score(y_test, dt.predict(X_test))
svm_train_recall = recall_score(y_train, svm.predict(X_train))
svm_test_recall = recall_score(y_test, svm.predict(X_test))

In [117]:
# Store in DF 
results_4['Model'] = ['Logistic Regression', 'K Neighbors', 'Decision Tree', 'SVM']
results_4['Train Time'] = [round(lr_train_time, 3), round(knn_train_time, 3), round(dt_train_time, 3), round(svm_train_time, 3)]
results_4['Train Recall'] = [round(lr_train_recall, 3), round(knn_train_recall, 3), round(dt_train_recall, 3), round(svm_train_recall, 3)]
results_4['Test Recall'] = [round(lr_test_recall, 3), round(knn_test_recall, 3), round(dt_test_recall, 3), round(svm_test_recall, 3)]

In [119]:
results_4 = results_4.set_index('Model')
results_4

,Train Time,Train Recall,Test Recall
Model,,,
Logistic Regression,0.230,0.233,0.229
K Neighbors,0.006,0.397,0.285
Decision Tree,0.160,0.965,0.338
SVM,23.250,0.267,0.242
